In [1]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [9]:
driver = webdriver.Chrome()
driver.set_page_load_timeout(15)
driver.get('https://sellercentral.amazon.com/forums/c/account-health')

for i in range(30):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

data_subcat = driver.find_elements_by_css_selector('span[class = "category-name"]')
sub_cats = [x.text for x in data_subcat[2:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content_tempo = [x.text.replace('\n', ' ') for x in data]
    contents.append(content_tempo)

time.sleep(5)
driver.quit()

In [10]:
posts = []
responses = []
for line in contents:
    posts.append(line[0])
    responses.append(line[1:])

In [11]:
def create_response_df(urls, resps):
    column_names = ['URL', 'Response Text']
    df_resps = pd.DataFrame(columns = column_names)
    for url,line in zip(urls, resps):
        n = len(line)
        sub_urls = [url] * n
        new_df = pd.DataFrame(list(zip(sub_urls, line)), columns = column_names)
        df_resps = pd.concat([df_resps, new_df])
    return df_resps

In [17]:
DF1 = pd.DataFrame(titles, columns=['Title'])
DF1['Category'] = 'Account Health'
DF1['Sub-category'] = sub_cats
DF1['Post Text'] = posts
DF1['URL'] = links

In [18]:
DF1.head()

,Title,Category,Sub-category,Post Text,URL
0,Account Suspended After 5 years out of nowhere...,Account Health,Related Accounts,Our brand was suspended about 25 minutes ago. ...,https://sellercentral.amazon.com/forums/t/acco...
1,How to handle potential trademark misuse dings,Account Health,Intellectual Property,Hi. I got some potential trademark misuse in m...,https://sellercentral.amazon.com/forums/t/how-...
2,Unfounded Suspected IP Violations,Account Health,Intellectual Property,I had a listing removed by bot (not a complain...,https://sellercentral.amazon.com/forums/t/unfo...
3,3 Asin suspend - pls seller performance team a...,Account Health,Product Authenticity,Hi. It was the new account. I sent a total of ...,https://sellercentral.amazon.com/forums/t/3-as...
4,Amazon Covid 19 Bot Has Gotten Ridiculous,Account Health,Restricted Products,We have been selling dietary supplements on Am...,https://sellercentral.amazon.com/forums/t/amaz...


In [19]:
DF2 = create_response_df(links, responses)

In [20]:
DF2.head()

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/acco...,Do you have any other accounts using your addr...
1,https://sellercentral.amazon.com/forums/t/acco...,Have the same issue here. Account was deactiva...
2,https://sellercentral.amazon.com/forums/t/acco...,We just got suspended too for the same reason....
3,https://sellercentral.amazon.com/forums/t/acco...,"Not at all, there’s only 1 Amazon account."
4,https://sellercentral.amazon.com/forums/t/acco...,"That’s exactly our problem, how the hell are w..."


In [21]:
DF1.shape

(900, 5)

In [25]:
DF1.to_csv('DF1.csv', index = False)
DF2.to_csv('DF2.csv', index = False)